In [1]:
""" Example 1:  A simple combination logic block example.
    This example declares a block of hardware with three one-bit inputs,
    (a,b,c) and two one-bit outputs (sum, cout).  The logic declared is a
    simple one-bit adder and the definition uses some of the most common
    parts of PyRTL. The adder is then simulated on random data, the
    wave form is printed to the screen, and the resulting trace is
    compared to a "correct" addition.  If the result is correct then a 0
    is returned, else 1.
"""

import random
import sys
sys.path.insert(0, './PyRTL')

import pyrtl
# import PyRTL

# The basic idea of PyRTL is to specify the component of a some hardware block
# through the declaration of wires and operations on those wires.  The current
# working block, an instance of a class devilishly named "Block", is implicit
# in all of the below code -- it is easiest to start with the way wires work.

# --- Step 1: Define Logic -------------------------------------------------

# One of the most fundamental types in PyRTL is the "WireVector" which is acts
# very much like a python list of 1-bit wires.  Unlike a normal list though the
# number of bits is explicitly declared.
temp1 = pyrtl.WireVector(bitwidth=1, name='temp1')

# Both arguments are in fact optional and default to a bitwidth of 1 and a unique
# name generated by pyrtl starting with 'tmp'
temp2 = pyrtl.WireVector()

# Two special types of WireVectors are Input and Output, which are used to specify
# an interface to the hardware block.
a, b, c = pyrtl.Input(1, 'a'), pyrtl.Input(1, 'b'), pyrtl.Input(1, 'c')
sum, carry_out = pyrtl.Output(1, 'sum'), pyrtl.Output(1, 'carry_out')

# Okay, let's build a one-bit adder.  To do this we need to use the assignment
# operator, which is '<<='.  This takes an already declared wire and "connects"
# it to some other already declared wire.  Let's start with the sum bit, which is
# of course just the xor of the three inputs
sum <<= a ^ b ^ c

# The carry_out bit would just be "carry_out <<= a & b | a & c | b & c" but let's break
# than down a bit to see what is really happening.  What if we want to give names
# to the partial signals in the middle of that computation.  When you take
# "a & b" in PyRTL what that really means is "make an AND gate, connect one input
# to 'a' and the other to 'b' and return the result of the gate".  The result of
# that AND gate can then be assigned to temp1 or it can be used like any other
# python variable.

temp1 <<= a & b  # connect the result of a & b to the pre-allocated wirevector
temp2 <<= a & c
temp3 = b & c  # temp3 IS the result of b & c (this is the first mention of temp3)
carry_out <<= temp1 | temp2 | temp3

# You can access the working block through pyrt.working_block(), and for most
# things one block is all you will need.  Example 2 discusses this in more detail,
# but for now we can just bring the block to see that in fact it looks like the
# hardware we described.  The format is a bit weird, but roughly translates to
# a list of gates (the 'w' gates are just wires).  The ins and outs of the gates
# are printed 'name'/'bitwidth''WireVectorType'

print('--- One Bit Adder Implementation ---')
print(pyrtl.working_block())
print()

# --- Step 2: Simulate Design  -----------------------------------------------

# Okay, let's get simulate our one-bit adder.  To keep track of the output of
# the simulation we need to make a new "SimulationTrace" and a "Simulation"
# that then uses that trace.

sim_trace = pyrtl.SimulationTrace()
sim = pyrtl.Simulation(tracer=sim_trace)

# Now all we need to do is call "sim.step" to simulate each clock cycle of our
# design.  We just need to pass in some input each cycle which is a dictionary
# mapping inputs (the *names* of the inputs, not the actual Input instances)
# and a value for that signal each cycle.  In this simple example we
# can just specify a random value of 0 or 1 with python's random module.  We
# call step 15 times to simulate 15 cycles.

for cycle in range(3):
    sim.step({
        'a': random.choice([0,1]),
        'b': random.choice([0,1]),
        'c': random.choice([0,1])
        })

# Now all we need to do is print the trace results to the screen. Here we use
# "render_trace" with some size information.
print('--- One Bit Adder Simulation ---')
sim_trace.render_trace(symbol_len=5, segment_size=5)

a_value = sim.inspect(a)
print("The latest value of a was: " + str(a_value))

# --- Verification of Simulated Design ---------------------------------------

# Now finally, let's check the trace to make sure that sum and carry_out are actually
# the right values when compared to a python's addition operation.  Note that
# all the simulation is done at this point and we are just checking the wave form
# but there is no reason you could not do this at simulation time if you had a
# really long running design.

for cycle in range(3):
    # Note that we are doing all arithmetic on values NOT wirevectors here.
    # We can add the inputs together to get a value for the result
    add_result = (sim_trace.trace['a'][cycle] +
                  sim_trace.trace['b'][cycle] +
                  sim_trace.trace['c'][cycle])
    # We can select off the bits and compare
    python_sum = add_result & 0x1
    python_cout = (add_result >> 1) & 0x1
    if (python_sum != sim_trace.trace['sum'][cycle] or
       python_cout != sim_trace.trace['carry_out'][cycle]):
        print('This Example is Broken!!!')
        exit(1)

# You made it to the end! 

exit(0)


--- One Bit Adder Implementation ---
tmp1/1W <-- ^ -- a/1I, b/1I 
tmp3/1W <-- & -- a/1I, b/1I 
tmp5/1W <-- & -- b/1I, c/1I 
tmp4/1W <-- & -- a/1I, c/1I 
tmp2/1W <-- ^ -- tmp1/1W, c/1I 
temp1/1W <-- w -- tmp3/1W 
sum/1O <-- w -- tmp2/1W 
tmp0/1W <-- w -- tmp4/1W 
tmp6/1W <-- | -- temp1/1W, tmp0/1W 
tmp7/1W <-- | -- tmp6/1W, tmp5/1W 
carry_out/1O <-- w -- tmp7/1W 

--- One Bit Adder Simulation ---


AttributeError: 'str' object has no attribute 'name'

In [ ]:
import random
import sys
sys.path.insert(0, './PyRTL')

import pyrtl
from pyrtl import *

# --- Part 1: Memories -------------------------------------------------------

# Memories is a way to store multiple sets of data for extended periods of
# time. Below we will make two instances of the same memory to test using
# that the same thing happens to two different memories using the same
# inputs

mem1 = MemBlock(bitwidth=32, addrwidth=3, name='mem')
mem2 = MemBlock(32, 3, 'mem')

# One memory will receive the write address from an input, the other, a register
waddr = Input(3, 'waddr')
count = Register(3, 'count')

# In order to make sure that the two memories take the same inputs,
# we will use same write data, write enable, and read addr values
wdata = Input(32, 'wdata')
we = Input(1, 'we')
raddr = Input(3, 'raddr')

# We will be grabbing data from each of the two memory blocks so we need
# two different output wires to see the results

rdata1 = Output(32, 'rdata1')
rdata2 = Output(32, 'rdata2')

# Ports
# The way of sending data to and from a memory block is through the
# use of a port. There are two types of ports, read ports and write ports.
# Each memory can have multiple read and write ports, but it doesn't make
# sense for one to have either 0 read ports or 0 write ports. Below, we
# will make one read port for each of the two memories

rdata1 <<= mem1[raddr]
rdata2 <<= mem2[raddr]

# Write Enable Bit
# For the write ports, we will do something different. Sometimes you don't
# want the memories to always accept the data and address on the write port.
# The write enable bit allows us to disable the write port as long as the
# value is zero, giving us complete control over whether to accept the data.

WE = MemBlock.EnabledWrite
mem1[waddr] <<= WE(wdata, we)  # Uses input wire
mem2[count] <<= WE(wdata, we)  # Uses count register

# Now we will finish up the circuit
# We will increment count register on each write

count.next <<= select(we, falsecase=count, truecase=count + 1)

# we will also verify that the two write address are always the same

validate = Output(1, 'validate')
validate <<= waddr == count

# Now it is time to simulate the circuit. first we will set up the values
# for all of the inputs.
# Write 1 through 8 into the eight registers, then read back out
simvals = {
    'we':        "00111111110000000000000000",
    'waddr':     "00012345670000000000000000",
    'wdata':     "00123456789990000000000000",
    'raddr':     "00000000000000000123456777"
}

# for simulation purposes, we can give the spots in memory an initial value
# note that in the actual circuit, the values are initially undefined
# below, we are building the data with which to initialize memory
mem1_init = {addr: 9 for addr in range(8)}
mem2_init = {addr: 9 for addr in range(8)}

# The simulation only recognizes initial values of memories when they are in a
# dictionary composing of memory : mem_values pairs.
memvals = {mem1: mem1_init, mem2: mem2_init}

# now run the simulation like before. Note the adding of the memory
# value map.
sim_trace = pyrtl.SimulationTrace()
sim = pyrtl.Simulation(tracer=sim_trace, memory_value_map=memvals)
for cycle in range(len(simvals['we'])):
    sim.step({k: int(v[cycle]) for k, v in simvals.items()})
sim_trace.render_trace()

# cleanup in preparation for the rom example
pyrtl.reset_working_block()

# --- Part 2: ROMs -----------------------------------------------------------

# ROMs are another type of memory. Unlike normal memories, ROMs are read only
# and therefore only have read ports. They are used to store predefined data

# There are two different ways to define the data stored in the ROMs
# either through passing a function or though a list or tuple


def rom_data_func(address):
    return 31 - 2 * address

rom_data_array = [rom_data_func(a) for a in range(16)]

# Now we will make the ROM blocks. ROM blocks are similar to memory blocks
# but because they are read only, they also need to be passed in a set of
# data to be initialized as

# FIXME: rework how memassigns work to account for more read ports
rom1 = RomBlock(bitwidth=5, addrwidth=4, romdata=rom_data_func, max_read_ports=10)
rom2 = RomBlock(5, 4, rom_data_array, max_read_ports=10)

rom_add_1, rom_add_2 = Input(4, "rom_in"), Input(4, "rom_in_2")

rom_out_1, rom_out_2 = Output(5, "rom_out_1"), Output(5, "rom_out_2")
rom_out_3, cmp_out = Output(5, "rom_out_3"), Output(1, "cmp_out")

# Because output wirevectors cannot be used as the source for other nets,
# in order to use the rom outputs in two different places, we must instead
# assign them to a temporary variable.

temp1 = rom1[rom_add_1]
temp2 = rom2[rom_add_1]

rom_out_3 <<= rom2[rom_add_2]

# now we will connect the rest of the outputs together

rom_out_1 <<= temp1
rom_out_2 <<= temp2

cmp_out <<= temp1 == temp2

# One of the things that is useful to have is repeatability, However, we
# also don't want the hassle of typing out a set of values to test. One
# solution in this case is to seed random and then pulling out 'random'
# numbers from it.

random.seed(4839483)

# Now we will create a new set of simulation values. In this case, since we
# want to use simulation values that are larger than 9 we cannot use the
# trick used in previous examples to parse values. The two ways we are doing
# it below are both valid ways of making larger values

simvals = {
    'rom_in': [1, 11, 4, 2, 7, 8, 2, 4, 5, 13, 15, 3, 4, 4, 4, 8, 12, 13, 2, 1],
    'rom_in_2': [random.randrange(0, 16) for i in range(20)]
}

# Now run the simulation like before. Note that for ROMs, we do not
# supply a memory value map because ROMs are defined with the values
# predefined.

sim_trace = pyrtl.SimulationTrace()
sim = pyrtl.Simulation(tracer=sim_trace)
for cycle in range(len(simvals['rom_in'])):
    sim.step({k: v[cycle] for k, v in simvals.items()})
sim_trace.render_trace()